In [5]:
import numpy as np
import pandas as pd
import keras 
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import matplotlib.pyplot as plt


In [11]:
def segment(frame):
    y = np.array(frame['PERCENTAGE'])
    X = np.array(frame.drop(['Unnamed: 0','ACC_DATE','MAINROAD_NAME','PERCENTAGE'], axis = 1))
    
    for i in range(0,X.shape[1]):
        X[:,i] = (X[:,i] - X[:,i].mean())/(X[:,i].max() - X[:,i].min())
        
    for i in range(y.shape[0]):
        y[i] = (y[i] - y.mean())/(y.max() - y.min())
    return X, y
                 
df_train = pd.read_csv('Train_bitcamp.csv')
X, y = segment(df_train)
X

array([[ 0.11551232,  0.12973418, -0.02673716, -0.25758673],
       [ 0.03217899, -0.08765713,  0.0006601 , -0.25758673],
       [ 0.03217899, -0.00070061,  0.07737243, -0.25758673],
       ...,
       [ 0.03217899, -0.08765713, -0.41851799, -0.05758673],
       [ 0.03217899,  0.12973418,  0.08833133, -0.05758673],
       [ 0.03217899,  0.2166907 ,  0.411619  , -0.05758673]])

In [14]:
model = Sequential()

'''
Inputs:
    -Speed limit INT (30- 70)
    -hour @day INT   (0 - 24)
    -Weather @ hour  (0 - 12)
    -date INT        (0 -365)
    
    -rural/urban BOOLEAN
    
Outputs:

   cars crashed @hour@day@MPH
---------------------------------
   car usage @hour@MPH (this is generalized for the whole year)
   
'''

#Make sure to normalize data
#y is target data, crash or not
#X is all the perameters

model.add(Dense(20, activation = 'relu', input_shape = (4,)))
for x in range(0,10):
    model.add(Dense(200, activation = 'relu'))
    #model.add(Dropout(0.2))
    
model.add(Dense(1, activation = 'sigmoid'))

In [21]:
optimizer = SGD(lr = .005, decay = 1e-6, momentum = 0.9)
model.compile(optimizer = optimizer, loss = 'mean_absolute_error')
model.fit(X, y, epochs = 20, validation_split = 0.30)



Train on 78670 samples, validate on 33717 samples
Epoch 1/20
78670/78670 [==============================] - 13s 163us/step - loss: 0.0038 - val_loss: 0.0030
Epoch 2/20
75584/78670 [===========================>..] - ETA: 0s - loss: 0.0037

KeyboardInterrupt: 

In [19]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [62]:
x = np.array([[.8,.4,.5,.5]])
x.shape

(1, 4)

In [63]:
model.predict(x, batch_size=None, verbose=0, steps=None)[0][0]

4.1183757e-05